In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [2]:
from loadFile import getPathList, fileToDataFrame

# ディレクトリ 内のメールファイルを読み込む
directory_path = "enron/maildir/allen-p/inbox"
file_list, dir_list=getPathList(directory_path)

# テキストファイルをデータフレームに格納する
df = fileToDataFrame(file_list)

In [3]:
#ドキュメント（メッセージ）をセンテンスに分解する
from preProcess import toSentences

sent_dict = {'Message-ID':[],
            'Sentence-ID':[],
            'Sentence':[]}

start = time.time()

for i,message in enumerate(df['Message']):
    sentences = toSentences(message)
    sent_dict['Message-ID'].extend([df['Message-ID'][i]] * len(sentences))
    sent_dict['Sentence-ID'].extend(range(len(sentences)))
    sent_dict['Sentence'].extend(sentences)

sentence_df = pd.DataFrame.from_dict(sent_dict,orient='columns')

end = time.time()
print(end-start,'s')

0.09408187866210938 s


In [5]:
# トリプルテーブルを作る

from tripleExtraction import tripleExtraction

triple_dict = {'Message-ID':[],
             'Sentence-ID':[],
             'Triple':[]}

start = time.time()

counter = 0
for i, sentence in enumerate(sentence_df['Sentence']):
    triples = tripleExtraction(sentence)
    triple_dict['Message-ID'].extend([sentence_df['Message-ID'][i]] * len(triples))
    triple_dict['Sentence-ID'].extend([sentence_df['Sentence-ID'][i]] * len(triples))
    triple_dict['Triple'].extend(triples)
    
triple_df = pd.DataFrame.from_dict(triple_dict,orient='columns')
        
end = time.time()
print(end-start,'s')

393.9632821083069 s


In [6]:
'''
output = pd.merge(df[['Path','Message-ID','Subject']],pd.merge(sentence_df,triple_df,on=['Message-ID','Sentence-ID']),on='Message-ID')
output.to_csv('output/allen_p_inbox_triple2.csv')

output = pd.read_csv('output/allen_p_inbox_triple2.csv')
'''

In [10]:
import spacy 
nlp = spacy.load("en_core_web_sm")
for sentence in sentence_df['Sentence']:
    doc = nlp(sentence)
    for entity in doc.ents:
        print((entity.text, entity.label_))

('20', 'CARDINAL')
('Oct. 30, 2001', 'DATE')
('TVA', 'ORG')
('Tennessee Valley Authority', 'ORG')
('TVA', 'ORG')
('2000', 'DATE')
('Boulder', 'GPE')
('RDI POWERdat', 'ORG')
('Mexican', 'NORP')
('Dependence', 'GPE')
('Utilities', 'ORG')
('ASP', 'ORG')
('Boosting', 'ORG')
('the SPR Calls', 'ORG')
('Nabors', 'ORG')
('Command Drilling', 'ORG')
('Georgia Power', 'ORG')
('Nigerian', 'NORP')
('R.I.', 'GPE')
('Enron', 'ORG')
('Dabhol', 'GPE')
('Mexico', 'GPE')
('next decade', 'DATE')
('Boston', 'GPE')
('LNG', 'ORG')
('CMS', 'ORG')
('LNG', 'ORG')
('400,000', 'CARDINAL')
('today', 'DATE')
('Executive News', 'ORG')
('2001 - Platts', 'DATE')
('All Rights Reserved Market Brief', 'WORK_OF_ART')
('Monday, October 29', 'DATE')
('Stocks Close Change % Change', 'ORG')
('9,269.50', 'CARDINAL')
('275.7', 'CARDINAL')
('-2.89%', 'PERCENT')
('DJ 15', 'LAW')
('296.10', 'CARDINAL')
('-0.22%', 'PERCENT')
('NASDAQ 1,699.52', 'ORG')
('69.44', 'CARDINAL')
('-3.93%', 'PERCENT')
('1,077.48', 'DATE')
('26.3', 'CARDIN